#  <u> 1st step : Load

Open the final merge table of the 'extraction part '

In [20]:
# we import the useful librairies 

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import functools
import os
import plotly.express as px
from scipy.stats import shapiro
import ipywidgets as widgets
from ipywidgets import Layout
from ipywidgets import interact, interact_manual


# we open and read the merge table of all indicators
dataset = pd.read_csv ('./data/all_indicators_table.csv')
dataset = dataset.drop(dataset.columns[0], axis = 1)
dataset

,Code,Year,Deaths,LifeExpectancy,GenderInequality,GDP,Fertility,tertiary_education
0,AFG,1966,161659.0,35.5,NaN,500000224.0,7.3203,NaN
1,AFG,1967,162579.0,35.9,NaN,605919040.0,7.3387,NaN
2,AFG,1968,163573.0,36.4,NaN,572222720.0,7.3628,NaN
3,AFG,1969,164638.0,36.9,NaN,587037568.0,7.3887,NaN
4,AFG,1970,165430.0,37.4,NaN,728703360.0,7.3996,1.08
...,...,...,...,...,...,...,...,...
21407,ZWE,1945,NaN,NaN,NaN,41000000.0,NaN,NaN
21408,ZWE,1946,NaN,NaN,NaN,48000000.0,NaN,NaN
21409,ZWE,1947,NaN,NaN,NaN,59000000.0,NaN,NaN
21410,ZWE,1948,NaN,NaN,NaN,72000000.0,NaN,NaN


#  <u> 2nd step : INTEGRATION,
I start filter the datas, first by country so I made groups of similar countries
    
I choose to make some groups according to their income. I made 4 groups : 

    - Low-income economies (1,135 dollars or less)  

    - Lower-middle-income economies (1,136 to 4,465 dollars)    

    - Upper-middle-income economies (4,466 to  13,845 dollars) 

    - High-income economies (13,845 dollars or more)  

In [21]:
low_income = ['AFG', 'BDI', 'CAF', 'COG', 'GMB', 'LBR', 'MWI', 'MLI', 'MOZ', 'NER', 'RWA', 'SLE', 'SDN', 'TGO', 'UGA', 'YEM'] 
lower_middle_income = ['DZA', 'BGD', 'BOL', 'KHM', 'CMR', 'COD', 'CIV', 'EGY', 'SWZ', 'GHA', 'HTI', 'HND', 'IDN', 'JOR', 'IRN', 'KEN', 'KGZ', 'LSO', 'MRT', 'MNG', 'MAR', 'NPL', 'NIC', 'NER', 'PAK', 'PNG', 'PHL', 'SEN', 'LKA', 'TJK', 'TZA', 'TUN', 'UKR', 'VNM', 'ZMB', 'ZWE']  
upper_middle_income = ['ALB', 'ARG', 'ARM', 'BLZ', 'BWA', 'BRA', 'BGR', 'CHN', 'COL', 'CRI', 'CUB', 'DOM', 'ECU', 'SLV', 'FJI', 'GAB', 'GTM', 'IDN', 'IRQ', 'JAM', 'KAZ', 'LBY', 'MYS', 'MDV', 'MUS', 'MEX', 'MDA', 'NAM', 'PRY', 'PER', 'ZAF', 'THA', 'TON'] 
high_income = ['AUS', 'AUT', 'BRB', 'BEL', 'CAN', 'CHL', 'HRV', 'CYP', 'DNK', 'EST', 'FIN', 'FRA', 'GUY', 'DEU', 'GRC', 'HUN', 'ISL', 'IRL', 'ISR', 'ITA', 'JPN', 'KWT', 'LVA', 'LTU', 'LUX', 'MLT', 'NLD', 'NZL', 'NOR', 'PAN', 'POL', 'PRT', 'QAT', 'ROU', 'SGP', 'ESP', 'SWE', 'CHE', 'TTO', 'ARE', 'GBR', 'USA', 'URY'] 

country_list=low_income+lower_middle_income+upper_middle_income+high_income

We reduce the dataset with only groups of levels of incomes, we can also filter by indicators or even by year if we reduce the min and max dates

However, our dataset is not so big so maybe we will first work on the merge dataset to have more datas before use the filtered dataset

In [22]:
dataset_countries=dataset.loc[dataset['Code'].isin(country_list)]
dataset_countries

,Code,Year,Deaths,LifeExpectancy,GenderInequality,GDP,Fertility,tertiary_education
0,AFG,1966,161659.0,35.5,NaN,500000224.0,7.3203,NaN
1,AFG,1967,162579.0,35.9,NaN,605919040.0,7.3387,NaN
2,AFG,1968,163573.0,36.4,NaN,572222720.0,7.3628,NaN
3,AFG,1969,164638.0,36.9,NaN,587037568.0,7.3887,NaN
4,AFG,1970,165430.0,37.4,NaN,728703360.0,7.3996,1.08
...,...,...,...,...,...,...,...,...
21407,ZWE,1945,NaN,NaN,NaN,41000000.0,NaN,NaN
21408,ZWE,1946,NaN,NaN,NaN,48000000.0,NaN,NaN
21409,ZWE,1947,NaN,NaN,NaN,59000000.0,NaN,NaN
21410,ZWE,1948,NaN,NaN,NaN,72000000.0,NaN,NaN


### So now we come back using the all_indicators_table and not the new one with the groups, we will do groups after 

Then we put all the indicators in only one column 

In [23]:
bronze_dataset=(dataset.set_index(["Code", "Year"]).stack().reset_index(name='Value').rename(columns={'level_2':'Indicator'}))
bronze_dataset

,Code,Year,Indicator,Value
0,AFG,1966,Deaths,1.616590e+05
1,AFG,1966,LifeExpectancy,3.550000e+01
2,AFG,1966,GDP,5.000002e+08
3,AFG,1966,Fertility,7.320300e+00
4,AFG,1967,Deaths,1.625790e+05
...,...,...,...,...
66664,ZWE,1945,GDP,4.100000e+07
66665,ZWE,1946,GDP,4.800000e+07
66666,ZWE,1947,GDP,5.900000e+07
66667,ZWE,1948,GDP,7.200000e+07


In [25]:
bronze_dataset.to_csv('./data/bronze_dataset.csv')